# Day 14: Docking Data

[*Advent of Code 2020 day 14*](https://adventofcode.com/2020/day/14) and [*solution megathread*](https://redd.it/kcr1ct)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2020/14/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2020%2F14%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


# Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [5]:
testdata = """mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0""".splitlines()

inputdata = downloaded['input'].splitlines()

In [6]:
from math import log2, ceil


def count_bits(arg):
    return sum([(arg >> b) & 1
                for b in range(ceil(log2(arg)) + 1)
                ])


def print_bits(arg):
    return "".join(map(str, [(arg >> b) & 1
                             for b in range(36, -1, -1)]
                       ))


def print_int(arg):
    return sum([
        (arg[-(b + 1)] == '1') * 2**b
        for b in range(len(arg))
    ])


def parse_mask(mask):
    positive, negative, floating = (0, 2**37 - 1, 0)
    mask = mask[::-1]
    for i in range(len(mask)):
        if mask[i] == '1':
            positive += 2**i
        elif mask[i] == '0':
            negative -= 2**i
        elif mask[i] == 'X':
            floating += 2**i
    return positive, negative, floating


def parse_instruction(line, debug: bool = False):
    a, b = line.split(' = ')
    if debug:
        print(line)
    if a == 'mask':
        return a, parse_mask(b)
    elif a[0:3] == 'mem':
        return 'mem', int(a[4:-1]), int(b)


def set_mask(p_n, debug: bool = False):
    global MASK_P
    global MASK_N
    global MASK_F
    MASK_P, MASK_N, MASK_F = p_n[0]
    if debug:
        print(f'Set masks to {MASK_P}, {MASK_N} and {MASK_F}')


def set_mem(addr_arg, debug: bool = False):
    addr, arg = (addr_arg[0], addr_arg[1])
    if debug:
        print(f'Set memory {addr} to {(arg | MASK_P ) & MASK_N}'
              f' (originally {arg}, with masks {MASK_P} and {MASK_N}')
    MEM[addr] = (arg | MASK_P) & MASK_N


opcodes = {
    'mask': set_mask,
    'mem': set_mem,
}


def execute(instructions, debug: bool = False):
    for instr in [parse_instruction(instruction, debug)
                  for instruction in instructions]:
        # print(f'executing: {instr}')
        opcodes[instr[0]](instr[1:], debug)

In [7]:
print(2**37 - 1)
print(count_bits(15))
print(print_bits(234567))
print(print_int('111001010001000111'))
print(parse_mask('XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X'))
print(parse_instruction('mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X'))
print(parse_instruction('mem[8] = 11'))
print(parse_instruction('mem[7] = 101'))

137438953471
4
0000000000000000000111001010001000111
234567
(64, 137438953469, 68719476669)
('mask', (64, 137438953469, 68719476669))
('mem', 8, 11)
('mem', 7, 101)


In [8]:
MASK_P, MASK_N = (0, 2**36 - 1)
MEM = dict()

execute(testdata, debug=True)
print(MEM, MASK_P, MASK_N)
print(f'Sum of memory: {sum([val for val in MEM.values()])}')

mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0
Set masks to 64, 137438953469 and 68719476669
Set memory 8 to 73 (originally 11, with masks 64 and 137438953469
Set memory 7 to 101 (originally 101, with masks 64 and 137438953469
Set memory 8 to 64 (originally 0, with masks 64 and 137438953469
{8: 64, 7: 101} 64 137438953469
Sum of memory: 165


In [9]:
MASK_P, MASK_N, MASK_F = (0, 2**37 - 1, 0)
MEM = dict()

execute(inputdata)
# print(MEM, MASK_P, MASK_N, MASK_F)
print(f'Sum of memory: {sum([val for val in MEM.values()])}')

Sum of memory: 13727901897109


In [10]:
HTML(downloaded['part1_footer'])

## Part Two

In [11]:
HTML(downloaded['part2'])

In [12]:
testdata2 = """mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1""".splitlines()

In [13]:
# This ain't gonna work, generates a ton of addresses?
def deduce_addrs(addr):
    addr = addr | MASK_P
    addrs = [addr]
    for b in range(37):
        if (MASK_F >> b) & 1:
            if (addr >> b) & 1:
                addrs += [a - 2**b for a in addrs]
            else:
                addrs += [a + 2**b for a in addrs]
    return addrs


def set_mem2(addr_arg, debug: bool = False):
    addr, arg = (addr_arg[0], addr_arg[1])
    for addr_f in deduce_addrs(addr):
        if debug:
            print(f'Set memory {print_bits(addr_f)} ({addr_f}) to {arg}'
                  f' (originally {addr}, with masks {MASK_P} and {MASK_N}')
        MEM[addr_f] = arg


opcodes2 = {
    'mask': set_mask,
    'mem': set_mem2,
}

In [14]:
def execute2(instructions, debug: bool = False):
    for instr in [parse_instruction(instruction, debug)
                  for instruction in instructions]:
        if debug:
            print(f'executing: {instr}')
        opcodes2[instr[0]](instr[1:])

In [15]:
MASK_P, MASK_N, MASK_F = (0, 2**37 - 1, 0)
MEM = dict()
execute2(testdata2, debug=True)
print(MEM, MASK_P, MASK_N, MASK_F)
print(f'Sum of memory: {sum([val for val in MEM.values()])}')

mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1
executing: ('mask', (18, 68719476787, 33))
executing: ('mem', 42, 100)
executing: ('mask', (0, 68719476747, 11))
executing: ('mem', 26, 1)
{58: 100, 59: 100, 26: 1, 27: 1, 24: 1, 25: 1, 18: 1, 19: 1, 16: 1, 17: 1} 0 68719476747 11
Sum of memory: 208


In [16]:
MASK_P, MASK_N, MASK_F = (0, 2**37 - 1, 0)
MEM = dict()
execute2(inputdata)
# print(MEM, MASK_P, MASK_N, MASK_F)
print(f'Sum of memory: {sum([val for val in MEM.values()])}')

Sum of memory: 5579916171823


In [17]:
HTML(downloaded['part2_footer'])